In [ ]:
import torch

torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#!pip install transformers numpy torch sklearn

In [ ]:
#!pip install sentence_transformers

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import random
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    max1 = 0
    max2 = 0
    for i in range(len(probs[0])):
        if probs[0][i] > probs[0][max1]:
            max2 = max1
            max1 = i
    if probs[0][max2] > 0.2:
        return {'top1 : ' : target_names[max1], 'top2 : ' : target_names[max2]}
    return target_names[max1]

def get_probs(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to('cuda')
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs

def get_embeds(text):
    emb = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to('cpu')
    return emb['input_ids']


def cleanText(text):
    text = text.lower()
            
    text = re.sub(r'[_\n/ _–<\*+«,\#+\№\"\-+\_+\=+{\?+\»%!+\&\}^\+\;\+\>«+"\(\)\/+\:\\+]', r' ', text)
    
    text = re.sub(r'[abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ]', r' ', text)
    
    text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', r' ', text)
    
    text = re.sub(r'(\d+\s\d+)|(\d+)',' ', text)
    
    text = re.sub(r'\s+', ' ', text)
    
    text = re.sub(r'\uf06c\uf020', ' ', text)

    return text


def get_words(model, text, size=5, how_many=5):
    n_gram_range = (size, size)
    count = CountVectorizer(ngram_range=n_gram_range).fit([text])
    candidates = count.get_feature_names()
    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)
    top_n = how_many
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {'accuracy': acc,
            'f1'      : f1}

In [ ]:
df = pd.read_csv('data.csv')

labels = {'Договоры оказания услуг' : 0,
         'Договоры купли-продажи'   : 1,
         'Договоры аренды'          : 2,
         'Договоры подряда'         : 3,
         'Договоры поставки'        : 4}

target_names = ['Договоры оказания услуг',
                'Договоры купли-продажи',
                'Договоры аренды',
                'Договоры подряда',
                'Договоры поставки']

df.label = df.label.replace(labels)
cleaned = []

for text in df.text:
    text = cleanText(text)
    cleaned.append(text)
df['clean'] = cleaned

In [ ]:
model_name = "sberbank-ai/ruBert-base"
max_length = 512

tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
transformer = SentenceTransformer(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
model.load_state_dict(torch.load('/content/doc_classifier/pytorch_model.bin'))
model.to('cuda')

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sberbank-ai--ruBert-base/snapshots/43be4261797042e172adf7476c558734f3cbb2a0/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--ruBert-base/snapshots/43be4261797042e172adf7476c558734f3cbb2a0/config.json
Model config BertConfig {
  "_name_or_path": "sberbank-ai/ruBert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_he

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
X_train, X_test, y_train, y_true = train_test_split(df['lemmatized'], df['label'], random_state=42, test_size=0.2, shuffle=True)
X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train.tolist()
y_true = y_true.tolist()
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsGroupsDataset(train_encodings, y_train)
valid_dataset = NewsGroupsDataset(test_encodings, y_true)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',        
    num_train_epochs=10,           
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=20,  
    warmup_steps=500,            
    weight_decay=0.01,           
    logging_dir='./logs',        
    load_best_model_at_end=True,     
    logging_steps=50,             
    save_steps=50,
    evaluation_strategy="steps", 
)

trainer = Trainer(
    model=model.to('cuda'),                 
    args=training_args,             
    train_dataset=train_dataset,    
    eval_dataset=valid_dataset,         
    compute_metrics=compute_metrics,     
)

using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

In [ ]:
model_path = "doc_classifier_new"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in doc_classifier/config.json
Model weights saved in doc_classifier/pytorch_model.bin
tokenizer config file saved in doc_classifier/tokenizer_config.json
Special tokens file saved in doc_classifier/special_tokens_map.json


('doc_classifier/tokenizer_config.json',
 'doc_classifier/special_tokens_map.json',
 'doc_classifier/vocab.txt',
 'doc_classifier/added_tokens.json',
 'doc_classifier/tokenizer.json')

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 91
  Batch size = 20


{'eval_loss': 0.36616000533103943,
 'eval_accuracy': 0.9120879120879121,
 'eval_f1': 0.9172882767234499,
 'eval_runtime': 2.6929,
 'eval_samples_per_second': 33.792,
 'eval_steps_per_second': 1.857,
 'epoch': 10.0}

In [ ]:
text = df.clean[10]

In [ ]:
get_words(transformer, text, 5, 5)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['наименование документа зарегистрированного наименование зарегистрировавшего',
 'выданного наименование органа выдавшего свидетельство',
 'полное наименование образовательного учреждения осуществляющего',
 'действующего основании наименование документа зарегистрированного',
 'наименование образовательного учреждения осуществляющего подготовку']